<a href="https://colab.research.google.com/github/wwh133/Transformer/blob/main/%E7%AC%AC7%E7%AB%A0_GPT_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



*   openAI gpt-3模型的训练包含17501个参数，在具有28500个CPU和10000个GPU的超级计算机上进行训练；
*   零样本实验：指不对模型的参数进行微调，直接将预训练模型用于下游，只需要使用API即可执行未专门训练的任务





*   GPT-2尚未达到基础模型的标准，基础模型不需要进行一步微调即可执行各种任务；
* GPT-3可以




Transformer模型从需要微调演化到零样本，**零样本的GPT-3 transformer模型不需要微调**



*   GPT-3模型，【层数】从6层增加到**96层**；
* GPT-3模型，【每层头数】从8个增加到**96个**
*   GPT-3模型，【上下文大小】从512个词元增加到**12288个**





> 上下文大小和最长距离

分析长期依赖关系所需的路径：从以前的 RNN、LSTM的循环层 转变为**注意力层**</br>
自注意力将操作简化为一对一的词源操作



*   RNN的循环层只能逐步存储上下文因此最长距离就是上下文大小
*   RNN不能将上下文拆分为在并行机器架构上运行的96个头




目标:使用无标注数据训练transformer模型让注意力层通过无监督数据来学习语言</br>
open AI训练transformer模型去学习语言,ta创建一个与任务无关的模型,使用原始数据来训练transformer模型而不是依赖专家标注的数据


*   open AI 先使用无监督学习来训练transformer模型,然后使用监督学习对模型进行微调
*   open AI 只使用原始transformer架构的解码器部分,效果不错




# 四个阶段


*   【微调】 先训练一个transformer模型,然后在下游任务上进行微调.——open AI团队逐步将微调任务数目减少到零
*   【少样本】gpt训练好后，需要推理时根据任务给出的少量样本**作为条件进行推理**，这种条件推理取代微调方式的权重更新，gpt这种方式就**不需要微调**了
* 【单样本】gpt训练好后，只需**一个样本作为条件进行推理**，而不需要微调更新权重
* 【零样本】gpt训练好后，**不需要任何样本**即可执行下游任务



# 预训练GPT-2 345M模型进行文本补全
包括24个解码器层、16个头的自注意力子层



In [8]:
#@title Step 2: Cloning the OpenAI GPT-2 Repository
#!git clone https://github.com/openai/gpt-2.git March 2023 update,changed to:
!git clone https://github.com/nshepperd/gpt-2

Cloning into 'gpt-2'...
remote: Enumerating objects: 429, done.
remote: Total 429 (delta 0), reused 0 (delta 0), pack-reused 429
Receiving objects: 100% (429/429), 4.47 MiB | 7.58 MiB/s, done.
Resolving deltas: 100% (218/218), done.


In [12]:
#@title Step 4 Checking the Version of TensorFlow
#Colab has tf 1.x and tf 2.x installed
#Restart runtime using 'Runtime' -> 'Restart runtime...'
#%tensorflow_version 1.x March 2023 update
import tensorflow as tf
print(tf.__version__)

2.15.0


In [10]:
#@title Step 5: Downloading the 345M parameter GPT-2 Model
# run code and send argument
import os # after runtime is restarted
os.chdir("/content/gpt-2")
!python3 download_model.py '345M'

Fetching checkpoint: 1.00kit [00:00, 4.14Mit/s]                                                     
Fetching encoder.json: 1.04Mit [00:00, 2.64Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 5.30Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:43, 32.6Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 18.4Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 2.41Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 1.74Mit/s]                                                       


In [14]:
#@title Step 7: Project Source Code
import os # import after runtime is restarted
os.chdir("/content/gpt-2/src")

In [19]:
#@title Step 7a: Interactive Conditional Samples (src)
#Project Source Code for Interactive Conditional Samples:
# /content/gpt-2/src/interactive_conditional_samples.py file
import json
import os
import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf #March 2023 update

In [15]:
#@title Step 7b: Importing model sample encoder
import model, sample, encoder
#if following message:
#ModuleNotFoundError: No module named 'tensorflow.contrib'
#then go back and run Step 2 Checking TensorFlow version

In [17]:
#@title Step 8: Defining the model：def interact_model
def interact_model(
    model_name, # 模型的名称，用于加载特定的 GPT-2 模型变体，如 '345M'。
    seed, # 生成随机数的种子，确保结果的可复现性
    nsamples, # 生成文本-样本数量
    batch_size, # 生成文本-批次大小
    length, # 生成文本-长度
    temperature, # 值越高生成的文本越随机
    top_k, # 从概率最高的 k 个单词中选择下一个单词
    models_dir # 模型文件的目录路径
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [ ]:
#@title Step 9: Interacting with GPT-2
interact_model('345M',None,1,1,300,1,0,'/content/gpt-2/models')

======================================== SAMPLE 1 ========================================
 If we can talk them down 60 beats, right? But then we have to blame ourselves. We say, well, it's halfspeed." Rex Kalen."

One day, I received a Spanish-speaking letter attached to my last receipt:   "... much to my joy, a Spanish mandolin today!! Thank you.""

My hope, Tom Layton of Eddy Scott Studio, goes out to all sax fans around the world when it is been discovered that their favorite saxophonist lost his eye (heinz lies buried next to him after an ugly accident there). He is now resting in Seychelles, avoiding flying with the tranquillity of an elated pilgrim.OK, it was 90 with me when I last saw Tom Layton, he is aging wallflower with good taste, and is unexpectedly taken with Italian venezia fragrantaria tobacco from erupting Mysteria Barbieri. Like much of his later work, it sounds clean, powerful, and melodious like it is straight out of the Ornette Coleman stylebox. It sounds ecstatic

# 自定义数据集，微调GPT-2 117M模型进行文本补全
包括12个解码器层、12个头的自注意力子层

In [ ]:
#@title Step 12: Interactive Context and Completion Examples 直接调用训练好的自定义gpt-2语言模型 进行文本补全
import os # import after runtime is restarted
os.chdir("/content/gpt-2/src")
!python interactive_conditional_samples.py --temperature 0.8 --top_k 40 --model_name '117M'

# 使用 零样本GPT-3 进行文本补全(在线运行)
https://platform.openai.com/playground

# 使用 GPT-3 进行文本补全（API接口）

## Step 1: Installing & importing OpenAI

In [3]:
try:
  import openai
except:
  !pip install openai
  import openai

## Step 2: Entering the API KEY
不支持给中国提供服务。。。。</br>
示例源代码：https://beta.openai.com/examples</br>
试验示例：https://platform.openai.com/playground

In [ ]:
openai.api_key="[YOUR_KEY]"

In [5]:
import os
import openai
os.environ['OPENAI_API_KEY'] ='[YOUR_KEY or KEY variable]'
print(os.getenv('OPENAI_API_KEY'))
openai.api_key = os.getenv("OPENAI_API_KEY")

[YOUR_KEY or KEY variable]


## Step 3: Running an NLP tasks with the default parameters
### Step 4: Example 1: Grammar correction

https://beta.openai.com/examples/default-grammar

In [ ]:
response = openai.Completion.create(
  engine="davinci",
  prompt="Original: She no went to the market.\nStandard American English:",
  temperature=0,
  max_tokens=60,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["\n"]
)

#displaying the response object
print(response)

# Fine-Tuning GPT-3/ 微调

Copyright 2022 Denis Rothman

[OpenAI fine-tuning documentation](https://beta.openai.com/docs/guides/fine-tuning/)



## 1.Preparing the dataset

### Step 1: Installing OpenAI & Wandb

Restart the runtime after installing openai and run the cell again to make sur that "import openai" is executed.

In [7]:
try:
  import openai
except:
  !pip install openai
  import openai

### Optional: Weights and Biases 【wandb可视化日志】



Use W&B to build better models faster. Track and visualize all the pieces of your machine learning pipeline, from datasets to production models.

Quickly identify model regressions. Use W&B to visualize results in real time, all in a central dashboard.
Focus on the interesting ML. Spend less time manually tracking results in spreadsheets and text files.
Capture dataset versions with W&B Artifacts to identify how changing data affects your resulting models.
Reproduce any model, with saved code, hyperparameters, launch commands, input data, and resulting model weights.


In [ ]:
try:
  import wandb
except:
  !pip install wandb
  import wandb

### Step 2: Your API Key

In [ ]:
openai.api_key=[YOUR_API_KEY]

### Step 3: Preparing the data

Answer "Y" to all of the questions.

In [ ]:
!openai tools fine_tunes.prepare_data -f "kantgpt.csv"

## 2.Fine-tuning ADA微调

### Step 4: Creating an OS environment for the API key

In [ ]:
import os
os.environ['OPENAI_API_KEY'] =API_KEY
print(os.getenv('OPENAI_API_KEY'))

### Step 5: Fine-tuning GPT-3 with the ADA engine

When prompted choose and click on the "ENTER" button.

In [ ]:
!openai api fine_tunes.create -t "kantgpt_prepared.jsonl" -m "ada"

### Step 6: Using the fine-tuned GPT-3 for a completion task

Note: If your fine-tuned model does not appear immediately after the end of the fine-tuning process, you might have to wait until it is processed by OpenAI. You can also:

1.go to the OpenAI Playground to test your model: https://platform.openai.com/playground

2.select your model in the dropdown list and test it in that environment

In [ ]:
!openai api completions.create -m ada:[YOUR_MODEL INFO] -p "Several concepts are a priori such as"